# Preparation

Preparation biasa dilakukan untuk mempersiapkan data sebelum masuk dalam tahap pemodelan. <br>
Berikut adalah tahapan yang akan dilalui pada data `SC_HW1_bank_data.csv` sebelum tahap pemodelan :
1. Import Library
2. Input Dataset
3. Preprocessing
4. Train-Test Split

## Import Library

In [ ]:
#mengimport library yang akan digunakan
import pandas as pd
import numpy as np

## Input Dataset

In [ ]:
#Membaca data dan memasukkannya ke dalam bentuk Pandas Dataframe
df = pd.read_csv('https://raw.githubusercontent.com/Rietaros/kampus_merdeka/main/SC_HW1_bank_data.csv')

In [ ]:
df.head()

,RowNumber,CustomerId,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
#Jalankan code untuk mengecek nama kolom yang tersedia
df.columns

Index(['RowNumber', 'CustomerId', 'Geography', 'Gender', 'Age', 'Tenure',
       'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
       'EstimatedSalary', 'Exited'],
      dtype='object')

In [ ]:
#Hilangkan kolom yang dirasa tidak relevan dengan model (contoh: ID). None dapat diisi dengan nama-nama kolom yang akan digunakan.
#Contoh df = df[['X1','X2', 'Y']].copy()

#START CODE
df = df[['Gender','Age','Tenure','Balance','NumOfProducts','HasCrCard','IsActiveMember','EstimatedSalary','Exited']].copy()

In [ ]:
df.head()

,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,Female,42,2,0.00,1,1,1,101348.88,1
1,Female,41,1,83807.86,1,0,1,112542.58,0
2,Female,42,8,159660.80,3,1,0,113931.57,1
3,Female,39,1,0.00,2,0,0,93826.63,0
4,Female,43,2,125510.82,1,1,1,79084.10,0


## Preprocessing

In [ ]:
#Lakukan One-Hot Encoder untuk data categorical, dengan fungsi pandas get_dummies

#START CODE
#df = pd.get_dummies(df,columns=['Gender','HasCrCard','IsActiveMember'])
df = pd.get_dummies(df)

In [ ]:
#Pisahkan mana X (feature) dengan Y,
#Y adalah kolom "Exited"

#START CODE
X = df.drop('Exited',axis=1)
y = df['Exited']

In [ ]:
#Lakukan Scaler dan/atau Normalisasi jika diperlukan
from sklearn.preprocessing import MinMaxScaler

#START CODE
scaler = MinMaxScaler()
X_transform = scaler.fit_transform(X)

In [ ]:
#Ini digunakan jika dilakukan scaler/Normalisasi. Jika tidak, code ini bisa dilewat dan diganti dengan code yang ada di dalam komen
X_transform = pd.DataFrame(X_transform, columns = X.columns)
#X_transform = X.copy()

## Train-Test Split

In [ ]:
#Split menjadi train dan test dengan test_size 25%
#Tidak perlu mengubah code ini

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_transform,y,test_size = 0.25,random_state = 123)

# Modeling

## Model1

### Soal :
Jelaskan secara Singkat Model pertama yang digunakan!

In [ ]:
# Naive Bayes. Supervised learning yang di mana dapat mengklasifikasikan objek dengan menetapkan label menggunakan probabilitas bersyarat
# probabilitas bersyarat yaitu peluang suatu peristiwa yang terjadi berdasarkan peristiwa lain yang telah terjadi.

from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)

y_pred = gnb.predict(X_test)
params_NB = {'var_smoothing': np.logspace(0,-9, num=100)} # menambahkan nilai ke dalam varians distribusi untuk 'menghaluskan'kurva dan menghasilkan-
#-lebih banyak sample yang semakin jauh dari rata-rata distribusi
#END CODE

#Lakukan parameter tuning sesuai hyperparameter yang dibutuhkan
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(
             estimator= gnb,
             param_grid= params_NB,
             scoring = 'accuracy',
             n_jobs = 10, # core cpu yang digunakan
             cv = 10 # 3-fold cross validation (artinya kita melakukan iterasi model sebanyak 3 kali)
            )

grid.fit(X_train,y_train)
grid.best_params_

{'var_smoothing': 0.01}

In [ ]:
#lakukan evaluasi dengan beberapa maetric di bawah ini
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

y_pred = grid.predict(X_test)

print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.98      0.90      1983
           1       0.73      0.22      0.34       517

    accuracy                           0.82      2500
   macro avg       0.78      0.60      0.62      2500
weighted avg       0.81      0.82      0.78      2500

[[1940   43]
 [ 403  114]]
0.8216


## Model2
### Soal :
Jelaskan secara Singkat Model ke-2 yang digunakan!

In [ ]:
# Random Forest. kombinasi dari decisiontree yang digabung sehingga membentuk suatu model baru. biasanya digunakan pada data set berjumlah besar.
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [ ]:
# execute nya 3 menitan:(
param_grid = {
    'n_estimators': [25, 50, 100, 150], #menentukan seberapa banyak 'pohon' keputusan yang ada. walaupun semakin baik, tetapi semakin besar nilainya bisa memperlama proses training
    'max_features': ['sqrt', 'log2', None], #membantu menemukan jumlah fitur yang perlu dipertimbangkan untuk membuat split yang baik
    'max_depth': [3, 6, 9], #jumlah split yang bisa dilakukan oleh setiap pohon keputusan. menentukan kedalaman maksimum tiap pohon keputusan
    'max_leaf_nodes': [3, 6, 9], #menetapkan kondisi split pada pohon keputusan
}

grid_search = GridSearchCV(RandomForestClassifier(),
                           param_grid=param_grid)
grid_search.fit(X_train, y_train)
print(grid_search.best_estimator_)

RandomForestClassifier(max_depth=6, max_features=None, max_leaf_nodes=9,
                       n_estimators=50)


In [ ]:
model_grid = RandomForestClassifier(max_depth=9,
                                    max_features="log2",
                                    max_leaf_nodes=9,
                                    n_estimators=25)
model_grid.fit(X_train, y_train)
y_pred_grid = model.predict(X_test)
print(classification_report(y_pred_grid, y_test))
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.87      0.91      2183
           1       0.45      0.73      0.55       317

    accuracy                           0.85      2500
   macro avg       0.70      0.80      0.73      2500
weighted avg       0.89      0.85      0.87      2500

[[1926   57]
 [ 305  212]]
0.8552


## Model3
### Soal :
Jelaskan secara Singkat Model ke-3 yang digunakan!

In [ ]:
# Decision Tree. terbentuk dari 1 node yang nantinya akan membentuk cabang-cabang pilihan yang baru hingga membentuk seperti sebuah pohon bercabang.
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(X_train,y_train)
params={'max_depth':[3,5,7,9,11,'max']} #menentukan kedalaman maksimum pohon keputusan

In [ ]:
#Lakukan parameter tuning sesuai hyperparameter yang dibutuhkan
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(
             estimator= dtc,
             param_grid= params,
             scoring = 'accuracy',
             n_jobs =10, # core cpu yang digunakan
             cv = 10, # 3-fold cross validation (artinya kita melakukan iterasi model sebanyak 3 kali)
            )

grid.fit(X_train,y_train)
grid.best_params_

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
10 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py", line 889, in fit
    super().fit(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py", line 177, in fit
    self._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 600, in _validat

{'max_depth': 5}

In [ ]:
#lakukan evaluasi
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

y_pred = grid.predict(X_test)

print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.97      0.91      1983
           1       0.79      0.41      0.54       517

    accuracy                           0.86      2500
   macro avg       0.83      0.69      0.73      2500
weighted avg       0.85      0.86      0.84      2500

[[1926   57]
 [ 305  212]]
0.8552


## Tarik Kesimpulan Model Mana yang terbaik beserta alasannya

In [ ]:
# Kesimpulannya, model terbaik yang bisa mendapatkan akurasi tertinggi yaitu menggunakan model Random Forest dan Decision Tree dengan nilai akurasi 86%.
# Karena dari fungsi nya sendiri yang bisa mem-breakdown komputasi yang kompleks menjadi sederhana, sehingga pengambilan keputusan menjadi lebih ter-interpretasi
# Penggunaan tuning hyperparameter dan scaler juga menjadi salah satu yang bisa membuat tingkat akurasinya meningkat, sehingga dibutuhkan dalam mem-prediksi "Churn status in Banking"